# NWAY demo

Author: Melissa

Date: Sep 25, 2025

This notebook demonstrates the use of the custom `NWAYCrossmatch` algorithm, implemented in the `lsdb-crossmatch` package.

We show it both in a traditional crossmatch, and a crossmatch-nested context.

In [1]:
import lsdb
import pandas as pd
from pathlib import Path

from lsdb_crossmatch.nway.nway_crossmatch import NWAYCrossmatch

pd.set_option("display.max_rows", None)

We have to use the `small` versions of the catalogs, since the larger versions crash my VSCode notebook.

In [2]:
catalog_dir = Path("./tests/data/m67")

small_ps1 = lsdb.open_catalog(catalog_dir / "ps1_cone_small")
small_delve = lsdb.open_catalog(catalog_dir / "delve_cone_small")

The `## Well beans` comment is in reference to not having a column in the DELVE catalog that represents the astrometric error, at all.

This is something that the NWAY algorithm expects, and it may be prudent to work with the NWAY folks to support missing fields. But not right now.

In [ ]:
xmatched = lsdb.crossmatch(
    small_ps1,
    small_delve,
    suffixes=("_ps1", "_delve"),
    algorithm=NWAYCrossmatch,
    radius_arcsec=60,
    left_mag_columns=["gMeanPSFMag", "rMeanPSFMag", "iMeanPSFMag", "zMeanPSFMag"],
    right_mag_columns=["MAG_PSF_G", "MAG_PSF_R", "MAG_PSF_I", "MAG_PSF_Z"],
    ra_error_col_left="raMeanErr",
    dec_error_col_left="decMeanErr",
    ra_error_col_right="MAGERR_PSF_G",  ## Well beans
    dec_error_col_right="MAGERR_PSF_I",
    match_flag=2,
).compute()

/home/delucchi/git/rsp/lsdb/src/lsdb/dask/crossmatch_catalog_data.py:147: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(
    adding angular separation columns
matching:  21304 matches after filtering by search radius
Primary catalogue "otmo" (939), density gives 3.79e+10 objects on entire sky
Catalogue "delve_dr2" (85), density gives 8.57e+08 objects on entire sky
Computing distance-based probabilities ...
    correcting for unrelated associations ...
100%|██████████| 939/939 [00:00<00:00, 4095.52it/s]

Computing final probabilities ...
    grouping by primary catalogue ID and flagging ...
/home/delucchi/.virtualenvs/sep/lib/python3.12/site-packages/nwaylib/__init__.py:457: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the

    adding angular separation columns
matching:  21304 matches after filtering by search radius
Primary catalogue "otmo" (939), density gives 3.79e+10 objects on entire sky
Catalogue "delve_dr2" (85), density gives 8.57e+08 objects on entire sky
Computing distance-based probabilities ...
    correcting for unrelated associations ...
100%|██████████| 939/939 [00:00<00:00, 6364.47it/s]

Computing final probabilities ...
    grouping by primary catalogue ID and flagging ...
/home/delucchi/.virtualenvs/sep/lib/python3.12/site-packages/nwaylib/__init__.py:457: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  table = table.groupby(table.columns[0], sort=False).apply(compute_group_statistics)


In [4]:
columns = ["objName_ps1", "QUICK_OBJECT_ID_delve"]
columns.extend(list(NWAYCrossmatch.extra_columns.columns))

xmatched[columns].head(10)

,objName_ps1,QUICK_OBJECT_ID_delve,Catalog_separation,Separation_max,ncat,dist_bayesfactor_uncorrected,dist_bayesfactor,dist_post,p_single,match_flag,prob_has_match,prob_this_match
_healpix_29,,,,,,,,,,,,
1387847909009725014,PSO J008.8601+11.7895,10480300015585,21.307281,21.307281,2,6.928542,6.928542,0.008726,0.008726,1,0.045915,0.182917
1387847909009725014,PSO J008.8601+11.7895,10480300045768,11.791379,11.791379,2,6.645164,6.645164,0.004563,0.004563,2,0.045915,0.095253
1387847909009725014,PSO J008.8601+11.7895,10480300045765,10.998684,10.998684,2,6.645372,6.645372,0.004565,0.004565,2,0.045915,0.095298
1387847909009725014,PSO J008.8601+11.7895,10480300015588,22.30418,22.30418,2,6.927579,6.927579,0.008707,0.008707,2,0.045915,0.182512
1387847909009725014,PSO J008.8601+11.7895,10480300015583,35.318123,35.318123,2,6.910965,6.910965,0.008383,0.008383,2,0.045915,0.175662
1387847909095340428,PSO J008.8611+11.7905,10480300015585,22.965116,22.965116,2,6.926925,6.926925,0.008694,0.008694,2,0.04985,0.167159
1387847909095340428,PSO J008.8611+11.7905,10480300045768,7.827015,7.827015,2,6.646049,6.646049,0.004572,0.004572,2,0.04985,0.08755
1387847909095340428,PSO J008.8611+11.7905,10480300045771,37.350627,37.350627,2,6.630969,6.630969,0.004417,0.004417,2,0.04985,0.084562
1387847909095340428,PSO J008.8611+11.7905,10480300045765,7.662537,7.662537,2,6.646081,6.646081,0.004573,0.004573,2,0.04985,0.087556


I hadn't thought of this until time to make this notebook (1:12 PM Thursday, BTW), but the `crossmatch_nested` routine also takes in a custom crossmatch. Since the NWAY algorithm loves to return all the possible matches, that makes it a nice candidate for this kind of nested matching.

In [5]:
xmatched_nested = small_ps1.crossmatch_nested(
    small_delve,
    nested_column_name="delve_matches",
    algorithm=NWAYCrossmatch,
    radius_arcsec=60,
    left_mag_columns=["gMeanPSFMag", "rMeanPSFMag", "iMeanPSFMag", "zMeanPSFMag"],
    right_mag_columns=["MAG_PSF_G", "MAG_PSF_R", "MAG_PSF_I", "MAG_PSF_Z"],
    ra_error_col_left="raMeanErr",
    dec_error_col_left="decMeanErr",
    ra_error_col_right="MAGERR_PSF_G",  ## Well beans
    dec_error_col_right="MAGERR_PSF_I",
    match_flag=2,
).compute()

/home/delucchi/git/rsp/lsdb/src/lsdb/dask/crossmatch_catalog_data.py:209: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


In [6]:
xmatched_nested["objName", "delve_matches"].head(10)

objName  \
_healpix_29                                  
1387847909009725014  PSO J008.8601+11.7895   
1387847909095340428  PSO J008.8611+11.7905   
1387847916069817131  PSO J008.8542+11.7915   
1387847916956866186  PSO J008.8551+11.7928   
1387847916956374328  PSO J008.8552+11.7927   
1387847917614728013  PSO J008.8548+11.7945   
1387847920086817580  PSO J008.8638+11.7901   
1387847920171439556  PSO J008.8634+11.7902   
1387847920238954418  PSO J008.8646+11.7902   
1387847920432284851  PSO J008.8637+11.7915   

                                                         delve_matches  
_healpix_29                                                             
1387847909009725014  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847909095340428  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847916069817131  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847916956866186  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847916956374328  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847917614728013  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847920086817580  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847920171439556  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847920238954418  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...  
1387847920432284851  [{QUICK_OBJECT_ID: 10480300015585, RA: 8.85509...